In [ ]:
"""
********************************** Neural Network **********************************
"""
"""
Hidden Layers use ReLU activation function, output layer is softmax activation function
and loss function is categorical cross entropy loss
"""

import numpy as np

class layer_dense:
  
  def __init__(self, n_inputs, n_neurons):
    self.weights = 0.01*np.random.randn(n_inputs, n_neurons)/np.sqrt(n_inputs/2)
    self.biases  = np.zeros((1, n_neurons))

  def forward(self, inputs):
    #remember inputs
    self.inputs = inputs
    self.output = np.dot(inputs, self.weights) + self.biases

  def backward(self, dvalues):
    self.dweights = np.dot(self.inputs.T, dvalues)
    self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
    self.dinputs = np.dot(dvalues, self.weights.T)
  
class Activation_ReLU:
  def forward(self, inputs):
    self.inputs = inputs
    self.output = np.maximum(0, inputs)

  def backward(self, dvalues):
    self.dinputs = dvalues.copy()
    self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
      self.inputs = inputs
      exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
      probabilities = exp_values/np.sum(exp_values, axis = 1, keepdims = True)
      self.output = probabilities

    def backward(self, dvalues):
      self.dinputs = np.empty_like(dvalues)
      for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
        single_output = single_output.reshape(-1, 1)
        jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
        self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

class Loss:
  def calculate(self, output, y):
    sample_losses = self.forward(output, y)
    data_loss = np.mean(sample_losses)
    return data_loss
  
class Loss_CategoricalCrossEntropy(Loss):

  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    elif len(y_true.shape) != 1:
      correct_confidences = np.sum(y_pred_clipped*y_true, axis = 1)

    negative_log_likelihoods = -np.log(correct_confidences)
    return negative_log_likelihoods

  def backward(self, dvalues, y_true):
    samples = len(dvalues)
    labels = len(dvalues[0])
    if len(y_true.shape) == 1:
      y_true = np.eye(labels)[y_true]
    self.dinputs = -y_true/dvalues
    self.dinputs = self.dinputs/samples

class Activation_Softmax_Loss_CategoricalCrossEntropy():

  def __init__ (self):
    self.activation = Activation_Softmax()
    self.loss = Loss_CategoricalCrossEntropy()

  def forward(self, inputs, y_true):
    self.activation.forward(inputs)
    self.output = self.activation.output
    return self.loss.calculate(self.output, y_true)

  def backward(self, dvalues, y_true):
    samples = len(dvalues)
    if len(y_true.shape) != 1:
      y_true = np.argmax(y_true, axis = 1)
    self.dinputs = dvalues.copy()
    self.dinputs[range(samples), y_true] -= 1
    self.dinputs = self.dinputs/samples

class Optimizer:

  def __init__(self, learning_rate = 1.0):
    self.learning_rate = learning_rate

  def update_params(self, layer):
    layer.weights -= self.learning_rate*layer.dweights
    layer.biases -= self.learning_rate*layer.dbiases

"""
********************************** Getting Images **********************************
"""

import matplotlib.pyplot as plt
from keras.datasets import cifar10
import cv2
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X = []  #stores 20 cat and dog images
y = []  #stores the label associated with each image

"""
function that gets n images of a particular class
and stores it in X and the labels in y
"""
def get_images(class_index, n):
  count = 0;
  i = 0
  if class_index == 3:
    label = 0
  elif class_index == 5:
    label = 1
  while(count < n):
    if(y_train[i] == class_index):
      X.append(X_train[i])
      y.append(label)
      count += 1
    i += 1

"""
In CIFAR10, cats are labeled 3 and dogs are labeled 5
Here cats will belong to class 0 and dogs to class 1
"""
get_images(3, 10) #adding 10 images of cats to X
get_images(5, 10) #adding 10 images of dogs to X

#convert the images into grayscale
X = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X])

"""
function that displays n images belonging to class "cls"
"""
def displayImages(n, cls):
  d = 0
  if cls == 'dog':
    d = 10
  elif cls == 'cat':
    d = 0
  fig, *a = plt.subplots(1, n)
  for i in range(n):
    a[0][i].imshow(X[i + d], cmap = 'gray')
    a[0][i].axis('off')

displayImages(3, 'cat')
displayImages(3, 'dog')
plt.show()

"""
********************************** Preprocessing **********************************
"""
print("\n\n\n")

#Converting the lists into numpy arrays
#Coverting 32*32 image into 1024 array
X = np.array(X, dtype = 'f').reshape(20, 32*32)
y = np.array(y)

#normalizing the data
for i in range(X.shape[0]):
  X[i] = (X[i] - np.mean(X[i]))/np.std(X[i])


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 2)

"""
********************************** Training **********************************
"""

layer1 = layer_dense(len(X_train[0]), 100)
activation1 = Activation_ReLU()
layer2 = layer_dense(100, 100)
activation2 = Activation_ReLU()
layer3 = layer_dense(100, 2)
output_layer = Activation_Softmax_Loss_CategoricalCrossEntropy()

optimizer = Optimizer(learning_rate = 0.1)    

layer1.forward(X_train)                      
activation1.forward(layer1.output)                        
layer2.forward(layer1.output)
activation2.forward(layer2.output) 
layer3.forward(activation2.output)                      
loss = output_layer.forward(layer3.output, Y_train)

fig, *a = plt.subplots(1, 2)
a[0][0].hist(activation1.output.reshape(1, -1)[0], bins = 50)
a[0][1].hist(activation2.output.reshape(1, -1)[0], bins = 50)
plt.show()

err = []
acc = []

plt.figure()

for epoch in range(500):
  layer1.forward(X_train)                      
  activation1.forward(layer1.output)                        
  layer2.forward(layer1.output)
  activation2.forward(layer2.output) 
  layer3.forward(activation2.output)                      
  loss = output_layer.forward(layer3.output, Y_train)

  predictions = np.argmax(output_layer.output, axis = 1)
  if len(Y_train.shape) != 1:
    Y_train = np.argmax(y, axis = 1)
  accuracy = np.mean(predictions == Y_train)

  if not epoch % 1:
    #print('loss: ', loss)
    err.append(loss)
    acc.append(accuracy)

  #Backward pass
  output_layer.backward(output_layer.output, Y_train)
  layer3.backward(output_layer.dinputs)
  activation2.backward(layer3.dinputs)
  layer2.backward(activation2.dinputs)
  activation1.backward(layer2.dinputs)
  layer1.backward(activation1.dinputs)
  
  optimizer.update_params(layer1)
  optimizer.update_params(layer2)
  optimizer.update_params(layer3)

print("\n\n")
#plt.figure()
plt.scatter(range(len(err)), err)
plt.scatter(range(len(acc)), acc)
plt.title('model loss')
plt.xlabel('epoch')
plt.legend(['loss','acc'], loc = 'lower left')
plt.show()
print("\ntraining accurarcy: ", acc[-1])

"""
********************************** Testing **********************************
"""
print("\n\n")

layer1.forward(X_test)                      
activation1.forward(layer1.output)                        
layer2.forward(layer1.output)
activation2.forward(layer2.output) 
layer3.forward(activation2.output)
#output_layer.forward(layer3.output, Y_test)
#test_predictions = np.argmax(output_layer.activation.output, axis = 1)
#print(test_predictions)
#Output layer
exp_values = np.exp(layer3.output - np.max(layer3.output, axis = 1, keepdims = True))
probabilities = exp_values/np.sum(exp_values, axis = 1, keepdims = True)

test_predictions = np.argmax(probabilities, axis = 1)

test_accuracy = np.mean(test_predictions == Y_test)

print('test_predictions: ', test_predictions)
print('test_labels: ', Y_test)

print('test data accuracy: ',test_accuracy)
